# Twin Cities Neighborhood Analysis

What I want to do.
1. Calculate the distance of all of them in relation to the airport and Oronoco.
2. Verify the miles calculation by spot checking a few with Google Maps
3. Find a way to overlay information (e.g. distance) on the shapes or popup.
3. Pull in other metadata (e.g. real estate, crime, water, etc...)

## Projection Summary
The city shapes are provided by the US Census Bureau in a geographic CRS NAD83 (authority code: [EPSG:4269](https://epsg.io/4269)). 
A _geographic_ projection is one in which that the coordinates are in latitude and longitude.
The US Census Bureau uses NAD83 for most of their stuff. This is an un-projected datum.
See this [article](https://source.opennews.org/articles/choosing-right-map-projection/) for more details.

A _projected_ CRS is in meters, feet, kilometers etc. (Note: You can see the 
active projection with the _crs_ property on a _GeoDataFrame_ object.)

The provided CRS -- EPSG:4269 -- isn't good for doing distance calculations. We need to 
use a equi-distant projection. To enable accurate calculations we use 
EPSG:4087 with is the  WGS 84 / World Equidistant Cylindrical CRS. 
This projection's UOM is in meters.

The popular online mapping systems (e.g. Google, Bing, Apple) all use [EPSG:3857](https://epsg.io/3857).
The [Wikipedia page](https://en.wikipedia.org/wiki/Web_Mercator_projection) has 
details on the pros and cons of this projection.

**To Clarify**
1. The spatial shapes are loaded in EPSG:4269.
2. The geometry is re-projected to EPSG:4087 to enable calculations.
3. Once the data is ready to be rendered on a map it is projected again to EPSG:3857.

In [3]:
from IPython.display import display
import geopandas as gdf
import pandas as pd

shapes = gdf.read_file('./data/cb_2018_27_cousub_500k.zip')

In [6]:
METERS_TO_MILES = 1609
MAX_DISTANCE_TO_AP = 15

# Project the geospatial data to a projected CRS to enable distance calculations.
mn_counties_shapes = shapes.to_crs("EPSG:4087")

# Find the centroid of all the shapes.
mn_counties_shapes['centroid'] = mn_counties_shapes.geometry.centroid

# The international airport is in Fort Snelling.
airport = mn_counties_shapes.loc[shapes['NAME'] == 'Fort Snelling']
airport_point = airport.iat[0, 11] # Get the centroid of the airport.

# Find the centroid of the Oronoco city limits.
# Note: The city of Oronoco is in the county of Oronoco.
oronoco = mn_counties_shapes[(mn_counties_shapes['NAME'] == 'Oronoco') & (mn_counties_shapes['COUSUBFP'] == '48598') ]
oronoco_point = oronoco.iat[0, 11]

# Find the distanace of all the shapes to the airport.
# The distance calculation is done in meters which is then converted to miles.
# 1 meter = 0.000621371 miles
# Divide # meters by 1609 to get the # miles.
mn_counties_shapes['miles_to_ap'] = mn_counties_shapes['centroid'].distance(airport_point)/METERS_TO_MILES
mn_counties_shapes['miles_to_oronoco'] = mn_counties_shapes['centroid'].distance(oronoco_point)/METERS_TO_MILES

# Project to the Mercator CRS for visualization.
mn_counties_shapes = mn_counties_shapes.to_crs("EPSG:3857")

# Find all the areas that are withing the maximum distance to the airport.
# Also include the city of Oronoco.
close_to_ap = mn_counties_shapes[(mn_counties_shapes['miles_to_ap'] <= MAX_DISTANCE_TO_AP) | ((mn_counties_shapes['NAME'] == 'Oronoco') & (mn_counties_shapes['COUSUBFP'] == '48598'))]

print (f'Filtering by Maximum Distance to Airport: {MAX_DISTANCE_TO_AP}')
print(f'Viewing {len(close_to_ap.index)} neighborhoods.')

# Create an interactive map using the Folium library.
close_to_ap.explore('miles_to_ap', legend=True)

# Note: I've got a bug with the current approach. By including Oronoco in the close_to_ap 
# dataframe, it's skewing the color scheme of miles_to_ap.


Filtering by Maximum Distance to Airport: 15
Viewing 37 neighborhoods.
